In [2]:
pip install googlemaps numpy shapely matplotlib pandas python-dotenv geojson


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install geopandas


  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 13.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import googlemaps
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point, MultiPolygon
import matplotlib.pyplot as plt
import geojson
from dotenv import load_dotenv
import time

# Load API key from .env file
load_dotenv('/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/API Keys/.env')
api_key = os.getenv('API_KEY')

# Initialize Google Maps client
gmaps = googlemaps.Client(key=api_key)

# Read the Whole Foods locations CSV
csv_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Ohio_Whole_Foods_Locations.csv'
locations_df = pd.read_csv(csv_path)

# Helper function to chunk a list into smaller lists
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

# List to hold isochrone polygons
isochrones = []

for index, row in locations_df.iterrows():
    central_point = (row['latitude'], row['longitude'])
    
    # Generate grid of points around the central point
    points = []
    for i in range(-5, 6):  # 11x11 grid to reduce the number of points
        for j in range(-5, 6):
            lat_offset = i * 0.01
            lng_offset = j * 0.01
            points.append((central_point[0] + lat_offset, central_point[1] + lng_offset))

    isochrone_points = []
    # Chunk points into smaller batches
    for point_chunk in chunk_list(points, 25):  # Chunk size of 25 to stay within limits
        # Request travel times from the central point to each chunk of grid points
        results = gmaps.distance_matrix(origins=[central_point], destinations=point_chunk, mode='driving')
        
        # Process results to extract travel times
        for i, element in enumerate(results['rows'][0]['elements']):
            travel_time = element['duration']['value'] / 60  # Convert seconds to minutes
            if travel_time <= 20:
                isochrone_points.append(point_chunk[i])
                
        # Add a small delay to avoid hitting rate limits
        time.sleep(1)
    
    # Convert isochrone points to Shapely Point objects
    shapely_points = [Point(lng, lat) for lat, lng in isochrone_points]

    # Create a convex hull around the points to form an approximate polygon
    if shapely_points:
        isochrone_polygon = Polygon(shapely_points).convex_hull
        isochrones.append(isochrone_polygon)

# Combine all isochrones into a single MultiPolygon
multi_isochrone = MultiPolygon(isochrones)

# Save the MultiPolygon to a GeoJSON file
geojson_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio/WF_20_Min_DT_Ohio.geojson'
geojson_feature = geojson.Feature(geometry=multi_isochrone, properties={})
with open(geojson_path, 'w') as f:
    geojson.dump(geojson_feature, f)

print(f"Isochrone polygons saved as {geojson_path}")

# Optional: Save as Shapefile using GeoPandas
import geopandas as gpd

gdf = gpd.GeoDataFrame(geometry=[multi_isochrone])
shapefile_path = '/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio/WF_20Min_DT_Ohio_Google.shp'
gdf.to_file(shapefile_path)

print(f"Isochrone polygons saved as {shapefile_path}")


Isochrone polygons saved as /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio/WF_20_Min_DT_Ohio.geojson
Isochrone polygons saved as /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project/GitHub Repo/OSU-AI-Bootcamp-Group-Project-1-Group-4/Resources/Shapefiles/WF_20_Min_DT_Ohio/WF_20Min_DT_Ohio_Google.shp


/opt/anaconda3/lib/python3.11/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
